In [1]:
# To support both python 2 and python 3
# 让这份笔记同步支持 python 2 和 python 3
from __future__ import division
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

In [2]:
#使用statsmodels便于统计分析数据
import statsmodels.api as sm
import statsmodels.formula.api as smf
#使用sklearn便于预测（机器学习）
from sklearn import linear_model
#使用patsy便于生成模型
import patsy

**6. We continue to consider the use of a logistic regression model to
predict the probability of default using income and balance on the
Default data set. In particular, we will now compute estimates for
the standard errors of the income and balance logistic regression co-
efficients in two different ways: (1) using the bootstrap, and (2) using
the standard formula for computing the standard errors in the glm()
function. Do not forget to set a random seed before beginning your
analysis.
(a) Using the summary() and glm() functions, determine the esti-
mated standard errors for the coefficients associated with income
and balance in a multiple logistic regression model that uses
both predictors.**

In [3]:
Default = pd.read_csv("data/Default.csv").drop('Unnamed: 0',axis = 1)
Default.head()

default student      balance        income
0      No      No   729.526495  44361.625074
1      No     Yes   817.180407  12106.134700
2      No      No  1073.549164  31767.138947
3      No      No   529.250605  35704.493935
4      No      No   785.655883  38463.495879

In [4]:
#将'default','student'哑变量便于分析
key_titles = ['default','student']
Default_Dummy = Default.copy()
for key_title in key_titles:
    dummies = pd.get_dummies(Default[key_title],prefix = key_title)
    Default_Dummy = Default_Dummy.join(dummies)
Default_Dummy.head()

default student      balance        income  default_No  default_Yes  \
0      No      No   729.526495  44361.625074           1            0   
1      No     Yes   817.180407  12106.134700           1            0   
2      No      No  1073.549164  31767.138947           1            0   
3      No      No   529.250605  35704.493935           1            0   
4      No      No   785.655883  38463.495879           1            0   

   student_No  student_Yes  
0           1            0  
1           0            1  
2           1            0  
3           1            0  
4           1            0

In [5]:
glm_smf = smf.logit('default_Yes ~ income + balance',data = Default_Dummy).fit()
glm_smf.summary()

Optimization terminated successfully.
         Current function value: 0.078948
         Iterations 10


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:            default_Yes   No. Observations:                10000
Model:                          Logit   Df Residuals:                     9997
Method:                           MLE   Df Model:                            2
Date:                Tue, 31 Dec 2019   Pseudo R-squ.:                  0.4594
Time:                        16:38:52   Log-Likelihood:                -789.48
converged:                       True   LL-Null:                       -1460.3
Covariance Type:            nonrobust   LLR p-value:                4.541e-292
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -11.5405      0.435    -26.544      0.000     -12.393     -10.688
income      2.081e-05   4.99e-06      4.174      0.000     1.1e-05    3.06e-05
balance        0.0056      0.000     24.835      0.000       0.005       0.006
==============================================================================

Possibly complete quasi-separation: A fraction 0.14 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""

**(b) Write a function, boot.fn() , that takes as input the Default data
set as well as an index of the observations, and that outputs
the coefficient estimates for income and balance in the multiple
logistic regression model.**

In [6]:
#> boot(Portfolio ,alpha.fn,R=1000)定义对data根据statistic统计方式采样R次的生成数据，详见实验
#> boot(Portfolio ,alpha.fn,R=1000)定义对data根据statistic统计方式采样R次的生成数据，详见实验
def bootstrap_stats(data, statistic, R):
    n_shape = len(data)
    try:
        statistic_original = list(statistic(data))
    except TypeError:
        statistic_original = [statistic(data)]
    statistic_list = []
    train_list = []
    statistic_len = len(statistic_original)
    for statistic_t in range(statistic_len):
        statistic_list.append([])
    for random_state in range(R):
        train = data.sample(n = n_shape,replace=True,random_state= random_state)
        try:
            float(statistic(train))
        except:
            try: 
                for statistic_factor in list(statistic(train)):
                    float(statistic_factor)
            except:
                continue
        train_list.append(train)
        for statistic_t in range(statistic_len):
            try:
                statistic_list[statistic_t].append(list(statistic(train))[statistic_t])
            except TypeError:
                statistic_list[statistic_t].append([statistic(train)][statistic_t])
    statistic_index = []
    statistic_mean = []
    statistic_std_error = []
    statistic_bias = []
    for statistic_t in range(statistic_len):
        statistic_index.append('t' + str(statistic_t) + '*')
        statistic_mean.append(np.mean(statistic_list[statistic_t]))
        statistic_std_error.append(((np.sum((statistic_list[statistic_t] - statistic_mean[statistic_t]) ** 2)) / (R - 1)) ** 0.5)
        statistic_bias.append(statistic_original[statistic_t] - statistic_mean[statistic_t])
    bootstrap_stats_pd = pd.DataFrame({'original':statistic_original,
                                       'mean':statistic_mean,
                                       'bias':statistic_bias,
                                       'std. error':statistic_std_error                                      
                                      },
                                     index = statistic_index)
    print(bootstrap_stats_pd)
    return statistic_list

In [7]:
def boot_fn(data):
    #return(smf.logit('default_Yes ~ income + balance',data).fit().params[[1,2]])
    return(smf.logit('default_Yes ~ income + balance',data).fit().params)
boot_fn(Default_Dummy)

Optimization terminated successfully.
         Current function value: 0.078948
         Iterations 10


Intercept   -11.540468
income        0.000021
balance       0.005647
dtype: float64

**(c) Use the boot() function together with your boot.fn() function to
estimate the standard errors of the logistic regression coefficients
for income and balance**

In [8]:
statistic_list = bootstrap_stats(Default_Dummy,boot_fn, 1000)

Optimization terminated successfully.
         Current function value: 0.078948
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.077812
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.077812
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.077812
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.077812
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.077812
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.078583
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.078583
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.078583
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.

Optimization terminated successfully.
         Current function value: 0.073501
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.073501
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.073501
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.073501
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.073501
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.082371
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.082371
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.082371
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.082371
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.

Optimization terminated successfully.
         Current function value: 0.078851
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.078851
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.078851
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.078851
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.078851
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.078761
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.078761
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.078761
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.078761
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.

Optimization terminated successfully.
         Current function value: 0.074953
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.074953
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.074953
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.074953
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.079647
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.079647
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.079647
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.079647
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.079647
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.

Optimization terminated successfully.
         Current function value: 0.078788
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.078788
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.078788
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.078788
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.078811
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.078811
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.078811
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.078811
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.078811
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.

...

Optimization terminated successfully.
         Current function value: 0.073877
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.081621
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.081621
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.081621
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.081621
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.081621
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.076858
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.076858
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.076858
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.

Optimization terminated successfully.
         Current function value: 0.077294
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.077294
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.083180
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.083180
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.083180
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.083180
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.083180
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.079775
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.079775
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.

Optimization terminated successfully.
         Current function value: 0.072784
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.072784
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.081971
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.081971
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.081971
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.081971
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.081971
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.082050
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.082050
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.

Optimization terminated successfully.
         Current function value: 0.076502
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.076502
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.076502
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.076502
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.082510
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.082510
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.082510
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.082510
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.082510
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.

Optimization terminated successfully.
         Current function value: 0.077826
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.077826
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.077826
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.077826
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.077826
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.074721
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.074721
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.074721
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.074721
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.

**(d) Now consider a logistic regression model that predicts the prob-
ability of default using income , balance , and a dummy variable
for student . Estimate the test error for this model using the val-
idation set approach. Comment on whether or not including a
dummy variable for student leads to a reduction in the test error
rate.**

income的标准误差基本一致，但balance的标准误差扩大了。

# 备注：GLM使用如下

In [9]:
glm_smf = smf.glm('default_Yes ~ income + balance',data = Default_Dummy, family = sm.families.Binomial()).fit()
glm_smf.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:            default_Yes   No. Observations:                10000
Model:                            GLM   Df Residuals:                     9997
Model Family:                Binomial   Df Model:                            2
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -789.48
Date:                Tue, 31 Dec 2019   Deviance:                       1579.0
Time:                        16:41:38   Pearson chi2:                 6.95e+03
No. Iterations:                     9                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -11.5405      0.435    -26.544      0.000     -12.393     -10.688
income      2.081e-05   4.99e-06      4.174      0.000     1.1e-05    3.06e-05
balance        0.0056      0.000     24.835      0.000       0.005       0.006
==============================================================================
"""